In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
import numpy as np
import pandas as pd
import tensorflow as tf
from gensim.models import KeyedVectors
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re

def clean_text(text):
    text = re.sub(r'https?://\S+', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^A-Za-z\s]', '', text)
    text = text.lower()
    return text

def vectorize_sentence(sentence, word_vectors, max_length=39):
    words = sentence.split()
    vectorized = [word_vectors[word] if word in word_vectors else np.zeros(300) for word in words[:max_length]]
    if len(vectorized) < max_length:
        vectorized.extend([np.zeros(300) for _ in range(max_length - len(vectorized))])
    return np.array([vectorized])


def predict_sentiment(texts, word_vectors, model):
    labels = ['Negative', 'Neutral', 'Positive']
    vectorized_texts = np.vstack([vectorize_sentence(text, word_vectors) for text in texts])
    predictions = model.predict(vectorized_texts)
    predicted_class_indices = np.argmax(predictions, axis=1)
    predicted_labels = [labels[idx] for idx in predicted_class_indices]
    return predicted_labels

df_smile = pd.read_csv('/content/drive/MyDrive/smile-annotations-final.csv', names=['id', 'text', 'category'])
df_smile['text'] = df_smile['text'].apply(clean_text)
sentiment_map = {
    'angry': 'Negative', 'disgust': 'Negative', 'disgust|angry': 'Negative',
    'happy': 'Positive', 'happy|sad': 'Positive',
    'nocode': 'Neutral', 'not-relevant': 'Neutral',
    'sad': 'Negative', 'sad|angry': 'Negative', 'sad|disgust': 'Negative',
    'sad|disgust|angry': 'Negative', 'surprise': 'Neutral'
}
df_smile['category'] = df_smile['category'].map(sentiment_map)

df_smile = df_smile.dropna(subset=['category'])
print("Unique class labels:", df_smile['category'].unique())

Unique class labels: ['Neutral' 'Positive' 'Negative']


In [3]:
!gdown --id "0B7XkCwpI5KDYNlNUTTlSS21pQmM"

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM
From (redirected): https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM&confirm=t&uuid=356a57d0-99b1-49d1-bfcd-3a544aa1001a
To: /content/GoogleNews-vectors-negative300.bin.gz
100% 1.65G/1.65G [00:17<00:00, 92.8MB/s]


In [20]:

word_vectors = KeyedVectors.load_word2vec_format('/content/GoogleNews-vectors-negative300.bin.gz', binary=True)



In [24]:

model = load_model('/content/drive/MyDrive/PaperCNN.keras')


test_texts = [
    "I love this product! Absolutely wonderful!",
    "This is the worst experience I ever had.",
    "I think this is okay, not too bad but could be better.",
    "The two works in our free #DuccioCaro exhibition play with space, movement, and architecture.",
    "Merci pour le partage!",
    "Say what, mate?"
]
cleaned_texts = [clean_text(text) for text in test_texts]
predicted_sentiments = predict_sentiment(cleaned_texts, word_vectors, model)
for text, sentiment in zip(cleaned_texts, predicted_sentiments):
    print(f'Text: {text}\nPredicted Sentiment: {sentiment}\n')


1/1 [==============================] - 0s 107ms/step
Text: i love this product absolutely wonderful
Predicted Sentiment: Positive

Text: this is the worst experience i ever had
Predicted Sentiment: Positive

Text: i think this is okay not too bad but could be better
Predicted Sentiment: Positive

Text: the two works in our free ducciocaro exhibition play with space movement and architecture
Predicted Sentiment: Neutral

Text: merci pour le partage
Predicted Sentiment: Neutral

Text: say what mate
Predicted Sentiment: Neutral

